<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-주소-가져올-검색-리스트-생성" data-toc-modified-id="1.-주소-가져올-검색-리스트-생성-1">1. 주소 가져올 검색 리스트 생성</a></span></li><li><span><a href="#2.-카카오-API-활용---주소-검색" data-toc-modified-id="2.-카카오-API-활용---주소-검색-2">2. 카카오 API 활용 - 주소 검색</a></span></li><li><span><a href="#3.-서울-지역-제외한-모든-행-제거" data-toc-modified-id="3.-서울-지역-제외한-모든-행-제거-3">3. 서울 지역 제외한 모든 행 제거</a></span><ul class="toc-item"><li><span><a href="#3.1.-서울-이외-지역-잘못된-행-주소-(혹은-전철역명)-변경" data-toc-modified-id="3.1.-서울-이외-지역-잘못된-행-주소-(혹은-전철역명)-변경-3.1">3.1. 서울 이외 지역 잘못된 행 주소 (혹은 전철역명) 변경</a></span></li><li><span><a href="#3.2.-서울-이외-지역에-해당하는-행-제거" data-toc-modified-id="3.2.-서울-이외-지역에-해당하는-행-제거-3.2">3.2. 서울 이외 지역에 해당하는 행 제거</a></span></li><li><span><a href="#3.3.-서울-지역-잘못된-행-주소-(혹은-전철역명)-변경" data-toc-modified-id="3.3.-서울-지역-잘못된-행-주소-(혹은-전철역명)-변경-3.3">3.3. 서울 지역 잘못된 행 주소 (혹은 전철역명) 변경</a></span></li></ul></li><li><span><a href="#4.-위도,-경도-가져올-주소-리스트-생성" data-toc-modified-id="4.-위도,-경도-가져올-주소-리스트-생성-4">4. 위도, 경도 가져올 주소 리스트 생성</a></span></li><li><span><a href="#5.-카카오-API-활용---위도,-경도-검색" data-toc-modified-id="5.-카카오-API-활용---위도,-경도-검색-5">5. 카카오 API 활용 - 위도, 경도 검색</a></span></li><li><span><a href="#6.-환승역-여부-컬럼-추가" data-toc-modified-id="6.-환승역-여부-컬럼-추가-6">6. 환승역 여부 컬럼 추가</a></span></li></ul></div>

- 공공데이터 포털 - 서울특별시 노선별 지하철역 정보(신규)
https://data.seoul.go.kr/dataList/OA-15442/S/1/datasetView.do

In [1]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas
import re

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('data/서울특별시 노선별 지하철역 정보(신규).csv', encoding='cp949')
print(data.shape)
data.head()

(730, 5)


,전철역코드,전철역명,전철명명(영문),호선,외부코드
0,2622,망원,Mangwon,06호선,621
1,2623,합정,Hapjeong,06호선,622
2,2624,상수,Sangsu,06호선,623
3,2625,광흥창,Gwangheungchang,06호선,624
4,2626,대흥,Daeheung,06호선,625


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   전철역코드     730 non-null    object
 1   전철역명      730 non-null    object
 2   전철명명(영문)  730 non-null    object
 3   호선        730 non-null    object
 4   외부코드      729 non-null    object
dtypes: object(5)
memory usage: 28.6+ KB


In [4]:
data['전철역명'].unique()

array(['망원', '합정', '상수', '광흥창', '대흥', '버티고개', '약수', '청구', '신당', '군자',
       '강남구청', '학동', '종로3가', '둔촌동', '올림픽공원', '미사', '하남풍산', '응암', '역촌',
       '불광', '독바위', '연신내', '구산', '새절', '증산', '디지털미디어시티', '월드컵경기장', '마포구청',
       '논현', '반포', '고속터미널', '양재', '매봉', '도곡', '대치', '학여울', '대청', '일원',
       '수서', '남태령', '방화', '개화산', '김포공항', '송정', '마곡', '애오개', '서대문', '광화문',
       '내방', '강변', '잠실나루', '잠실', '잠실새내', '동대입구', '금호', '옥수', '압구정', '신사',
       '잠원', '교대', '남부터미널', '이수', '까치울', '부천종합운동장', '춘의', '금정', '부평구청',
       '암사', '천호', '강동구청', '석촌고분', '종합운동장', '삼성', '선릉', '역삼', '강남', '경복궁',
       '안국', '을지로3가', '공덕', '효창공원앞', '석촌', '송파나루', '한성백제', '둔촌오륜', '방배',
       '사당', '낙성대', '서울대입구', '봉천', '신림', '신대방', '구로디지털단지', '신도림', '문래',
       '영등포구청', '삼각지', '녹사평', '이태원', '한강진', '용답', '신정네거리', '지축', '구파발',
       '녹번', '홍제', '무악재', '독립문', '동묘앞', '창신', '사가정', '용마산', '중곡', '녹양',
       '어린이대공원', '을지로입구', '을지로4가', '동대문역사문화공원', '한양대', '뚝섬', '성수', '건대입구',
       '구의', '당산', '홍대입구', '신촌', '이대', '아현', '충정로', '남

- 서울역 -> 서울로 변경
- 4?19민주묘지 -> 4.19민주묘지로 변경

In [5]:
# 정규표현식으로 전철역명 변경
data['전철역명'] = data['전철역명'].apply(lambda x : re.sub('[역$]','',x))
data['전철역명'] = data['전철역명'].apply(lambda x : re.sub('[?]','.',x))

In [6]:
data[data['전철역명']=='서울']

,전철역코드,전철역명,전철명명(영문),호선,외부코드
241,0426,서울,Seoul Station,04호선,426
297,1251,서울,Seoul Station,경의선,P313
599,4201,서울,Seoul Station,공항철도,A01
723,0150,서울,Seoul Station,01호선,133


In [7]:
data[data['전철역명']=='4.19민주묘지']

,전철역코드,전철역명,전철명명(영문),호선,외부코드
640,4703,4.19민주묘지,April 19th National Cemetery,우이신설경전철,943


In [8]:
data['호선'].unique()

array(['06호선', '07호선', '05호선', '03호선', '04호선', '02호선', '08호선', '09호선',
       '01호선', '인천2호선', '경의선', '분당선', '경춘선', '경강선', '수인선', '인천선', '공항철도',
       '신분당선', '의정부경전철', '용인경전철', '우이신설경전철', '서해선', '김포도시철도'],
      dtype=object)

In [9]:
# 주소 검색이 용이하도록 전철역명, 호선 변경
data['전철역명'] = data['전철역명'].apply(lambda x : x + '역')
data['호선'] = data['호선'].apply(lambda x : x.replace('0',''))
data.head()

,전철역코드,전철역명,전철명명(영문),호선,외부코드
0,2622,망원역,Mangwon,6호선,621
1,2623,합정역,Hapjeong,6호선,622
2,2624,상수역,Sangsu,6호선,623
3,2625,광흥창역,Gwangheungchang,6호선,624
4,2626,대흥역,Daeheung,6호선,625


In [10]:
# 주소 검색용 컬럼 생성
data['검색키워드'] = data['전철역명'] + ' ' + data['호선']
data.head()

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드
0,2622,망원역,Mangwon,6호선,621,망원역 6호선
1,2623,합정역,Hapjeong,6호선,622,합정역 6호선
2,2624,상수역,Sangsu,6호선,623,상수역 6호선
3,2625,광흥창역,Gwangheungchang,6호선,624,광흥창역 6호선
4,2626,대흥역,Daeheung,6호선,625,대흥역 6호선


## 1. 주소 가져올 검색 리스트 생성

In [11]:
len(data['검색키워드'])

730

In [12]:
search_list = data['검색키워드'].tolist()
search_list

['망원역 6호선',
 '합정역 6호선',
 '상수역 6호선',
 '광흥창역 6호선',
 '대흥역 6호선',
 '버티고개역 6호선',
 '약수역 6호선',
 '청구역 6호선',
 '신당역 6호선',
 '군자역 7호선',
 '강남구청역 7호선',
 '학동역 7호선',
 '종로3가역 5호선',
 '둔촌동역 5호선',
 '올림픽공원역 5호선',
 '미사역 5호선',
 '하남풍산역 5호선',
 '응암역 6호선',
 '촌역 6호선',
 '불광역 6호선',
 '독바위역 6호선',
 '연신내역 6호선',
 '구산역 6호선',
 '새절역 6호선',
 '증산역 6호선',
 '디지털미디어시티역 6호선',
 '월드컵경기장역 6호선',
 '마포구청역 6호선',
 '논현역 7호선',
 '반포역 7호선',
 '고속터미널역 7호선',
 '양재역 3호선',
 '매봉역 3호선',
 '도곡역 3호선',
 '대치역 3호선',
 '학여울역 3호선',
 '대청역 3호선',
 '일원역 3호선',
 '수서역 3호선',
 '남태령역 4호선',
 '방화역 5호선',
 '개화산역 5호선',
 '김포공항역 5호선',
 '송정역 5호선',
 '마곡역 5호선',
 '애오개역 5호선',
 '서대문역 5호선',
 '광화문역 5호선',
 '내방역 7호선',
 '강변역 2호선',
 '잠실나루역 2호선',
 '잠실역 2호선',
 '잠실새내역 2호선',
 '동대입구역 3호선',
 '금호역 3호선',
 '옥수역 3호선',
 '압구정역 3호선',
 '신사역 3호선',
 '잠원역 3호선',
 '고속터미널역 3호선',
 '교대역 3호선',
 '남부터미널역 3호선',
 '이수역 7호선',
 '까치울역 7호선',
 '부천종합운동장역 7호선',
 '춘의역 7호선',
 '금정역 4호선',
 '부평구청역 7호선',
 '암사역 8호선',
 '천호역 8호선',
 '강동구청역 8호선',
 '석촌고분역 9호선',
 '종합운동장역 2호선',
 '삼성역 2호선',
 '선릉역 2호선',
 '삼역 2호선',
 '강남역 2호선',
 '교대역 2호선',


## 2. 카카오 API 활용 - 주소 검색

In [55]:
# 주소 검색 함수 정의
def address_search(address):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?&query=" + address + "&category_group_code=SW8"
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['road_address_name']]
        except:
            val = [0]
    return val

In [56]:
address_search('서울역 공항철도')   # 서울역 경의선 제외 모두 이상 -> 추후 변경 예정

['서울 강서구 마곡중앙로 지하 135']

In [57]:
address_search('서울역 4호선')

['서울 강북구 도봉로 지하 50']

In [58]:
address_search('서울역 경의선')

['서울 중구 통일로 1']

In [59]:
address_search('서울역 1호선')

['경기 화성시 떡전골로 97']

In [63]:
# 주소 검색 함수 반복 적용
list1 = []
for i in search_list:
    list1.append(address_search(i))

In [64]:
data2 = pd.DataFrame(list1, columns=['주소'])
data2.head()

,주소
0,서울 마포구 월드컵로 지하 77
1,서울 마포구 양화로 지하 55
2,서울 마포구 독막로 지하 85
3,서울 마포구 독막로 지하 165
4,서울 마포구 대흥로 지하 85


In [65]:
data3 = pd.concat([data, data2], axis=1)
print(data3.shape)
data3.head()

(730, 7)


,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
0,2622,망원역,Mangwon,6호선,621,망원역 6호선,서울 마포구 월드컵로 지하 77
1,2623,합정역,Hapjeong,6호선,622,합정역 6호선,서울 마포구 양화로 지하 55
2,2624,상수역,Sangsu,6호선,623,상수역 6호선,서울 마포구 독막로 지하 85
3,2625,광흥창역,Gwangheungchang,6호선,624,광흥창역 6호선,서울 마포구 독막로 지하 165
4,2626,대흥역,Daeheung,6호선,625,대흥역 6호선,서울 마포구 대흥로 지하 85


## 3. 서울 지역 제외한 모든 행 제거

In [68]:
data4 = data3[data3['주소'].str.contains('서울') == False]
print(data4.shape)
data4.head()

(345, 7)


,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
15,2563,미사역,Misa,5호선,555,미사역 5호선,경기 하남시 미사강변동로 지하 90
16,2564,하남풍산역,Hanam Pungsan,5호선,556,하남풍산역 5호선,경기 하남시 덕풍서로 지하 50
63,2753,까치울역,Kkachiul,7호선,751,까치울역 7호선,경기 부천시 길주로 지하 626
64,2754,부천종합운동장역,Bucheon Stadium,7호선,752,부천종합운동장역 7호선,경기 부천시 길주로 지하 502
65,2755,춘의역,Chunui,7호선,753,춘의역 7호선,경기 부천시 길주로 지하 502


In [67]:
data4['검색키워드'].unique()

array(['미사역 5호선', '하남풍산역 5호선', '까치울역 7호선', '부천종합운동장역 7호선', '춘의역 7호선',
       '금정역 4호선', '부평구청역 7호선', '지축역 3호선', '녹양역 1호선', '철산역 7호선',
       '광명사거리역 7호선', '장암역 7호선', '검단사거리역 인천2호선', '신흥역 8호선', '수진역 8호선',
       '모란역 8호선', '인천대공원역 인천2호선', '용산역 1호선', '구리역 경의선', '도농역 경의선',
       '양정역 경의선', '덕소역 경의선', '도심역 경의선', '팔당역 경의선', '운길산역 경의선', '양수역 경의선',
       '신원역 경의선', '국수역 경의선', '아신역 경의선', '오빈역 경의선', '양평역 경의선', '원덕역 경의선',
       '용문역 경의선', '지평역 경의선', '화전역 경의선', '강매역 경의선', '행신역 경의선', '능곡역 경의선',
       '곡산역 경의선', '풍산역 경의선', '일산역 경의선', '탄현역 경의선', '운정역 경의선', '금릉역 경의선',
       '월롱역 경의선', '파주역 경의선', '문산역 경의선', '임진강역 경의선', '갈매역 경춘선', '별내역 경춘선',
       '퇴계원역 경춘선', '사릉역 경춘선', '평내호평역 경춘선', '마석역 경춘선', '대성리역 경춘선',
       '청평역 경춘선', '상천역 경춘선', '가평역 경춘선', '굴봉산역 경춘선', '백양리역 경춘선', '강촌역 경춘선',
       '김유정역 경춘선', '남춘천역 경춘선', '백마역 경의선', '야당역 경의선', '금촌역 경의선', '금곡역 경춘선',
       '천마산역 경춘선', '봉명역 1호선', '쌍용역 1호선', '아산역 1호선', '배방역 1호선',
       '온양온천역 1호선', '신창역 1호선', '경마공원역 4호선', '대공원역 4호선', '과천역 4호선',
       '정부과천청사역 

- 검색이 잘못된 전철역명 : 용산역 1호선, 오류동역 1호선, 동대문사문화공원역 5호선 (오타 있음), 시청역 2호선, 서울역 1호선, 시청역 1호선


- 이름만 잘못된 전철역명 : 곡역 1호선 (역곡역으로 추정)

### 3.1. 서울 이외 지역 잘못된 행 주소 (혹은 전철역명) 변경

In [69]:
data3[data3['검색키워드']=='용산역 1호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
249,1003,용산역,Yongsan,1호선,135,용산역 1호선,광주 동구 녹동길 45


In [72]:
data3.loc[249, '주소'] = '서울 용산구 한강대로23길 55'

In [74]:
data3[data3['검색키워드']=='오류동역 1호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
409,1802,오류동역,Oryu-dong,1호선,144,오류동역 1호선,대전 중구 계룡로 지하 793


In [75]:
data3.loc[409, '주소'] = '서울 구로구 경인로20길 13'

In [77]:
data3[data3['검색키워드']=='동대문사문화공원역 5호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
491,2537,동대문사문화공원역,Dongdaemun History Culture Park,5호선,536,동대문사문화공원역 5호선,경기 동두천시 평화로 2539


In [78]:
data3.loc[491, '전철역명'] = '동대문역사문화공원역'
data3.loc[491, '검색키워드'] = '동대문역사문화공원역 5호선'
data3.loc[491, '주소'] = '서울 중구 마른내로 지하 162'

In [81]:
data3[data3['검색키워드']=='시청역 2호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
649,0201,시청역,City Hall,2호선,201,시청역 2호선,인천 남동구 구월로 지하 99


In [82]:
data3.loc[649, '주소'] = '서울 중구 서소문로 지하 127'

In [84]:
data3[data3['검색키워드']=='시청역 1호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
724,0151,시청역,City Hall,1호선,132,시청역 1호선,대전 서구 둔산중로 지하 55


In [85]:
data3.loc[724, '주소'] = '서울 중구 세종대로 지하 101'

In [87]:
data3[data3['검색키워드']=='서울역 공항철도']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
599,4201,서울역,Seoul Station,공항철도,A01,서울역 공항철도,서울 강서구 마곡중앙로 지하 135


In [88]:
data3.loc[599, '주소'] = '서울 용산구 청파로 378'

In [89]:
data3[data3['검색키워드']=='서울역 4호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
241,0426,서울역,Seoul Station,4호선,426,서울역 4호선,서울 강북구 도봉로 지하 50


In [90]:
data3.loc[241, '주소'] = '서울 용산구 한강대로 지하 392'

In [92]:
data3[data3['검색키워드']=='서울역 1호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
723,0150,서울역,Seoul Station,1호선,133,서울역 1호선,경기 화성시 떡전골로 97


In [93]:
data3.loc[723, '주소'] = '서울 중구 세종대로 지하 2'

In [94]:
data3[data3['검색키워드']=='곡역 1호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
410,1803,곡역,Yeokgok,1호선,146,곡역 1호선,대전 중구 계룡로 지하 793


In [95]:
data3.loc[410, '전철역명'] = '역곡역'
data3.loc[410, '검색키워드'] = '역곡역 1호선'
data3.loc[410, '주소'] = '경기 부천시 역곡로 1'

### 3.2. 서울 이외 지역에 해당하는 행 제거

In [96]:
data5 = data3[data3['주소'].str.contains('서울') == False]
print(data5.shape)
data5.head()

(339, 7)


,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
15,2563,미사역,Misa,5호선,555,미사역 5호선,경기 하남시 미사강변동로 지하 90
16,2564,하남풍산역,Hanam Pungsan,5호선,556,하남풍산역 5호선,경기 하남시 덕풍서로 지하 50
63,2753,까치울역,Kkachiul,7호선,751,까치울역 7호선,경기 부천시 길주로 지하 626
64,2754,부천종합운동장역,Bucheon Stadium,7호선,752,부천종합운동장역 7호선,경기 부천시 길주로 지하 502
65,2755,춘의역,Chunui,7호선,753,춘의역 7호선,경기 부천시 길주로 지하 502


In [99]:
data6 = data3.drop(data5.index, axis=0).reset_index(drop=True)
print(data6.shape)
data6.head()

(391, 7)


,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
0,2622,망원역,Mangwon,6호선,621,망원역 6호선,서울 마포구 월드컵로 지하 77
1,2623,합정역,Hapjeong,6호선,622,합정역 6호선,서울 마포구 양화로 지하 55
2,2624,상수역,Sangsu,6호선,623,상수역 6호선,서울 마포구 독막로 지하 85
3,2625,광흥창역,Gwangheungchang,6호선,624,광흥창역 6호선,서울 마포구 독막로 지하 165
4,2626,대흥역,Daeheung,6호선,625,대흥역 6호선,서울 마포구 대흥로 지하 85


In [100]:
data6['검색키워드'].unique()

array(['망원역 6호선', '합정역 6호선', '상수역 6호선', '광흥창역 6호선', '대흥역 6호선',
       '버티고개역 6호선', '약수역 6호선', '청구역 6호선', '신당역 6호선', '군자역 7호선',
       '강남구청역 7호선', '학동역 7호선', '종로3가역 5호선', '둔촌동역 5호선', '올림픽공원역 5호선',
       '응암역 6호선', '촌역 6호선', '불광역 6호선', '독바위역 6호선', '연신내역 6호선', '구산역 6호선',
       '새절역 6호선', '증산역 6호선', '디지털미디어시티역 6호선', '월드컵경기장역 6호선', '마포구청역 6호선',
       '논현역 7호선', '반포역 7호선', '고속터미널역 7호선', '양재역 3호선', '매봉역 3호선',
       '도곡역 3호선', '대치역 3호선', '학여울역 3호선', '대청역 3호선', '일원역 3호선', '수서역 3호선',
       '남태령역 4호선', '방화역 5호선', '개화산역 5호선', '김포공항역 5호선', '송정역 5호선',
       '마곡역 5호선', '애오개역 5호선', '서대문역 5호선', '광화문역 5호선', '내방역 7호선',
       '강변역 2호선', '잠실나루역 2호선', '잠실역 2호선', '잠실새내역 2호선', '동대입구역 3호선',
       '금호역 3호선', '옥수역 3호선', '압구정역 3호선', '신사역 3호선', '잠원역 3호선',
       '고속터미널역 3호선', '교대역 3호선', '남부터미널역 3호선', '이수역 7호선', '암사역 8호선',
       '천호역 8호선', '강동구청역 8호선', '석촌고분역 9호선', '종합운동장역 2호선', '삼성역 2호선',
       '선릉역 2호선', '삼역 2호선', '강남역 2호선', '교대역 2호선', '경복궁역 3호선', '안국역 3호선',
       '종로3가역 3호선', '을지로3가역 3호선', '공덕역 6호선',

- 이름이 잘못된 전철역명 : 촌역 6호선, 삼역 2호선, 동대문사문화공원역 2호선, 동대문사문화공원역 4호선

### 3.3. 서울 지역 잘못된 행 주소 (혹은 전철역명) 변경

In [103]:
data6[data6['검색키워드']=='촌역 6호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
16,2612,촌역,Yeokchon,6호선,611,촌역 6호선,서울 은평구 증산로 지하 477


In [104]:
data6.loc[16, '전철역명'] = '역촌역'
data6.loc[16, '검색키워드'] = '역촌역 6호선'
data6.loc[16, '주소'] = '서울 은평구 서오릉로 지하 63'

In [109]:
data6[data6['검색키워드']=='삼역 2호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
68,0221,삼역,Yeoksam,2호선,221,삼역 2호선,서울 강남구 테헤란로 지하 340


In [110]:
data6.loc[68, '전철역명'] = '역삼역'
data6.loc[68, '검색키워드'] = '역삼역 2호선'
data6.loc[68, '주소'] = '서울 강남구 테헤란로 지하 156'

In [111]:
data6[data6['검색키워드']=='동대문사문화공원역 2호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
115,0205,동대문사문화공원역,Dongdaemun History Culture Park,2호선,205,동대문사문화공원역 2호선,서울 중구 을지로 지하 178


In [112]:
data6.loc[115, '전철역명'] = '동대문역사문화공원역'
data6.loc[115, '검색키워드'] = '동대문역사문화공원역 2호선'
data6.loc[115, '주소'] = '서울 중구 을지로 지하 279'

In [113]:
data6[data6['검색키워드']=='동대문사문화공원역 4호선']

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소
158,0422,동대문사문화공원역,Dongdaemun History Culture Park,4호선,422,동대문사문화공원역 4호선,서울 중구 장충단로 지하 230


In [114]:
data6.loc[158, '전철역명'] = '동대문역사문화공원역'
data6.loc[158, '검색키워드'] = '동대문역사문화공원역 4호선'
data6.loc[158, '주소'] = '서울 중구 장충단로 지하 230'

- 검색이 잘못된 전철역명(환승) : 강남역 2호선, 강남역 신분당선, 강동역 5호선, 건대입구역 2호선, 건대입구역 7호선(주소가 서로 바뀜), 구로역 1호선, 신도림역 2호선, 신도림역 1호선, 노원역 4호선, 노원역 7호선, 도곡역 3호선, 동대문역 4호선, 동대문역 1호선, 동작역 4호선, 동작역 9호선, 마곡나루역 9호선, 망우역 경의선, 망우역 경춘선, 보문역 우이신설경전철, 불광역 6호선, 불광역 3호선, 사당역 2호선, 사당역 4호선, 석촌역 9호선, 성수역 2호선, 성신여대입구역 우이신설경전철, 신길역 1호선, 신내역 6호선, 신당역 6호선, 신당역 2호선, 양재역 신분당선, 여의도역 5호선, 여의도역 9호선, 오금역 5호선, 옥수역 경의선, 용산역 경의선, 이촌역 경의선, 잠실역 2호선, 중랑역 경의선, 중랑역 경춘선, 창동역 4호선, 창동역 1호선, 천호역 5호선, 충정로역 5호선, 합정역 6호선, 합정역 2호선 (주소가 서로 바뀜), 효창공원앞역 경의선, 디지털미디어시티역 경의선, 신도림역 2호선, 신도림역 1호선, 신설동역 우이신설경전철, 종로3가역 5호선, 종로3가역 3호선, 종로3가역 1호선, 홍대입구역 2호선, 회기역 경춘선, 회기역 경의선, 상봉역 경춘선, 왕십리역 경의선, 왕십리역 2호선, 청량리역 경의선, 청량리역 경춘선

In [212]:
data6.loc[69, '주소'] = '서울 강남구 강남대로 지하 396'
data6.loc[338, '주소'] = '서울 강남구 강남대로 지하 396'
data6.loc[199, '주소'] = '서울 강동구 천호대로 지하 1097'
data6.loc[120, '주소'] = '서울 광진구 아차산로 243'
data6.loc[218, '주소'] = '서울 광진구 능동로 지하 110'
data6.loc[278, '주소'] = '서울 구로구 구로중앙로 174'
data6.loc[90, '주소'] = '서울 구로구 새말로 지하 117-21'
data6.loc[236, '주소'] = '서울 구로구 경인로 688'
data6.loc[147, '주소'] = '서울 노원구 상계로 69-1'
data6.loc[300, '주소'] = '서울 노원구 동일로 지하 1409'
data6.loc[31, '주소'] = '서울 강남구 남부순환로 지하 2814'
data6.loc[157, '주소'] = '서울 종로구 율곡로 지하 308'
data6.loc[362, '주소'] = '서울 종로구 종로 지하 302'
data6.loc[228, '주소'] = '서울 동작구 현충로 257'
data6.loc[328, '주소'] = '서울 동작구 현충로 지하 220'
data6.loc[310, '주소'] = '서울 강서구 마곡중앙5로 지하 2'
data6.loc[261, '주소'] = '서울 중랑구 망우로55길 11-10'
data6.loc[274, '주소'] = '서울 중랑구 망우로55길 11-10'
data6.loc[350, '주소'] = '서울 성북구 보문로 지하 116'
data6.loc[17, '주소'] = '서울 은평구 통일로 지하 723-1'
data6.loc[101, '주소'] = '서울 은평구 통일로 지하 723-1'
data6.loc[83, '주소'] = '서울 동작구 남부순환로 지하 2089'
data6.loc[230, '주소'] = '서울 동작구 동작대로 지하 3'
data6.loc[77, '주소'] = '서울 송파구 백제고분로 지하 368'
data6.loc[119, '주소'] = '서울 성동구 아차산로 100'
data6.loc[349, '주소'] = '서울 성북구 동소문로 지하 102'
data6.loc[258, '주소'] = '서울 영등포구 영등포로 327'
data6.loc[373, '주소'] = '서울 중랑구 용마산로136길 33'
data6.loc[8, '주소'] = '서울 중구 다산로 지하 260'
data6.loc[116, '주소'] = '서울 중구 퇴계로 지하 431-1'
data6.loc[339, '주소'] = '서울 서초구 남부순환로 지하 2585'
data6.loc[185, '주소'] = '서울 영등포구 여의나루로 지하 40'
data6.loc[326, '주소'] = '서울 영등포구 의사당대로 지하 101'
data6.loc[206, '주소'] = '서울 송파구 오금로 지하 321'
data6.loc[239, '주소'] = '서울 성동구 동호로 21'
data6.loc[382, '주소'] = '서울 용산구 한강대로23길 55'
data6.loc[237, '주소'] = '서울 용산구 이촌로71길 42'
data6.loc[49, '주소'] = '서울 송파구 올림픽로 지하 265'
data6.loc[259, '주소'] = '서울 중랑구 중랑역로 9'
data6.loc[381, '주소'] = '서울 중랑구 중랑역로 9'
data6.loc[148, '주소'] = '서울 도봉구 마들로11길 77'
data6.loc[248, '주소'] = '서울 도봉구 마들로11길 77'
data6.loc[198, '주소'] = '서울 강동구 천호대로 지하 997'
data6.loc[129, '주소'] = '서울 서대문구 충정로 지하 28-1'
data6.loc[1, '주소'] = '서울 마포구 양화로 지하 45'
data6.loc[123, '주소'] = '서울 마포구 양화로 지하 55'
data6.loc[266, '주소'] = '서울 용산구 백범로 지하 287'
data6.loc[281, '주소'] = '서울 은평구 수색로 193'
data6.loc[90, '주소'] = '서울 구로구 새말로 지하 117-21'
data6.loc[236, '주소'] = '서울 구로구 경인로 688'
data6.loc[369, '주소'] = '서울 동대문구 왕산로 지하 1'
data6.loc[12, '주소'] = '서울 종로구 돈화문로11길 지하 26'
data6.loc[73, '주소'] = '서울 종로구 돈화문로 지하 30'
data6.loc[360, '주소'] = '서울 종로구 종로 지하 129'
data6.loc[124, '주소'] = '서울 마포구 양화로 지하 160'
data6.loc[272, '주소'] = '서울 동대문구 회기로 196'
data6.loc[389, '주소'] = '서울 동대문구 회기로 196'
data6.loc[273, '주소'] = '서울 중랑구 망우로 297'
data6.loc[240, '주소'] = '서울 성동구 왕십리광장로 17'
data6.loc[372, '주소'] = '서울 성동구 왕십리로 지하 300'
data6.loc[241, '주소'] = '서울 동대문구 왕산로 214'
data6.loc[271, '주소'] = '서울 동대문구 왕산로 214'

- 검색이 잘못된 전철역명(환승X) : 가양역 9호선, 가오리역 우이신설경전철, 개포동역 분당선, 고덕역 5호선, 공릉역 7호선, 구의역 2호선, 논현역 7호선, 대치역 3호선, 도봉역 1호선, 등촌역 9호선, 마곡역 5호선, 마포역 5호선, 면목역 7호선, 명일역 5호선, 목동역 5호선, 미아역 4호선, 반포역 7호선, 방이역 5호선, 방화역 5호선, 봉천역 2호선, 북한산우이역 우이신설경전철, 삼성역 2호선, 삼양역 우이신설경전철, 삼전역 9호선, 상계역 4호선, 상도역 7호선, 서대문역 5호선, 서빙고역 경의선, 서울숲역 분당선, 서초역 2호선, 송파역 8호선, 신림역 2호선, 신사역 3호선, 신촌역 경의선, 양원역 경의선, 영등포역 1호선, 용답역 2호선, 월계역 1호선, 응봉역 경의선, 일원역 3호선, 잠실새내역 2호선, 장지역 8호선, 정릉역 우이신설경전철, 중곡역 7호선, 증산역 6호선, 한남역 경의선, 행당역 5호선, 홍제역 3호선, 화곡역 5호선

In [461]:
data6.loc[332, '주소'] = '서울 강서구 양천로 지하 485'
data6.loc[344, '주소'] = '서울 강북구 삼양로 426'
data6.loc[253, '주소'] = '서울 강남구 개포로 지하 420'
data6.loc[203, '주소'] = '서울 강동구 고덕로 지하 253'
data6.loc[303, '주소'] = '서울 노원구 동일로 지하 1074'
data6.loc[121, '주소'] = '서울 광진구 아차산로 384-1'
data6.loc[26, '주소'] = '서울 강남구 학동로 지하 102'
data6.loc[32, '주소'] = '서울 강남구 남부순환로 지하 2952'
data6.loc[292, '주소'] = '서울 도봉구 도봉로170길 2'
data6.loc[313, '주소'] = '서울 강서구 공항대로 지하 529'
data6.loc[42, '주소'] = '서울 강서구 공항대로 지하 163'
data6.loc[187, '주소'] = '서울 마포구 마포대로 지하 33'
data6.loc[173, '주소'] = '서울 중랑구 면목로 지하 407'
data6.loc[202, '주소'] = '서울 강동구 양재대로 지하 1632'
data6.loc[179, '주소'] = '서울 양천구 오목로 지하 245'
data6.loc[151, '주소'] = '서울 강북구 도봉로 지하 198'
data6.loc[27, '주소'] = '서울 서초구 신반포로 지하 241'
data6.loc[205, '주소'] = '서울 송파구 양재대로 지하 1127'
data6.loc[38, '주소'] = '서울 강서구 금낭화로 지하 132'
data6.loc[86, '주소'] = '서울 관악구 남부순환로 지하 1721'
data6.loc[340, '주소'] = '서울 강북구 삼양로 지하 676'
data6.loc[66, '주소'] = '서울 강남구 테헤란로 지하 538'
data6.loc[388, '주소'] = '서울 강북구 삼양로 지하 259'
data6.loc[216, '주소'] = '서울 송파구 백제고분로 지하 189'
data6.loc[146, '주소'] = '서울 노원구 상계로 182'
data6.loc[132, '주소'] = '서울 동작구 상도로 지하 272'
data6.loc[44, '주소'] = '서울 종로구 통일로 지하 126'
data6.loc[238, '주소'] = '서울 용산구 서빙고로 238'
data6.loc[287, '주소'] = '서울 성동구 왕십리로 지하 77'
data6.loc[371, '주소'] = '서울 서초구 서초대로 지하 233'
data6.loc[355, '주소'] = '서울 송파구 송파대로 지하 354'
data6.loc[87, '주소'] = '서울 관악구 남부순환로 지하 1614'
data6.loc[55, '주소'] = '서울 강남구 도산대로 지하 102'
data6.loc[265, '주소'] = '서울 서대문구 신촌역로 30'
data6.loc[262, '주소'] = '서울 중랑구 송림길 147'
data6.loc[235, '주소'] = '서울 영등포구 경인로 846'
data6.loc[97, '주소'] = '서울 성동구 용답길 86'
data6.loc[246, '주소'] = '서울 노원구 월계로53길 40'
data6.loc[263, '주소'] = '서울 성동구 고산자로 123'
data6.loc[35, '주소'] = '서울 강남구 일원로 지하 121'
data6.loc[50, '주소'] = '서울 송파구 올림픽로 지하 140'
data6.loc[358, '주소'] = '서울 송파구 송파대로 지하 82'
data6.loc[348, '주소'] = '서울 성북구 아리랑로 지하 122'
data6.loc[110, '주소'] = '서울 광진구 능동로 지하 417'
data6.loc[22, '주소'] = '서울 은평구 증산로 지하 306'
data6.loc[375, '주소'] = '서울 용산구 독서당로6길 12-13'
data6.loc[191, '주소'] = '서울 성동구 행당로 지하 89'
data6.loc[103, '주소'] = '서울 서대문구 통일로 지하 440-1'
data6.loc[176, '주소'] = '서울 강서구 화곡로 지하 168'

## 4. 위도, 경도 가져올 주소 리스트 생성

In [463]:
len(data6['주소'])

391

In [465]:
address_list = data6['주소'].tolist()
address_list

['서울 마포구 월드컵로 지하 77',
 '서울 마포구 양화로 지하 45',
 '서울 마포구 독막로 지하 85',
 '서울 마포구 독막로 지하 165',
 '서울 마포구 대흥로 지하 85',
 '서울 중구 다산로 지하 38',
 '서울 중구 다산로 지하 115',
 '서울 중구 청구로 지하 77',
 '서울 중구 다산로 지하 260',
 '서울 광진구 천호대로 지하 550',
 '서울 강남구 학동로 지하 346',
 '서울 강남구 학동로 지하 180',
 '서울 종로구 돈화문로11길 지하 26',
 '서울 강동구 양재대로 지하 1369',
 '서울 송파구 양재대로 지하 1233',
 '서울 은평구 증산로 지하 477',
 '서울 은평구 서오릉로 지하 63',
 '서울 은평구 통일로 지하 723-1',
 '서울 은평구 불광로 지하 129-1',
 '서울 은평구 통일로 지하 849',
 '서울 은평구 연서로 지하 137-1',
 '서울 은평구 증산로 지하 400',
 '서울 은평구 증산로 지하 306',
 '서울 은평구 수색로 지하 175',
 '서울 마포구 월드컵로 지하 240',
 '서울 마포구 월드컵로 지하 190',
 '서울 강남구 학동로 지하 102',
 '서울 서초구 신반포로 지하 241',
 '서울 서초구 신반포로 지하 188',
 '서울 서초구 남부순환로 지하 2585',
 '서울 강남구 남부순환로 지하 2744',
 '서울 강남구 남부순환로 지하 2814',
 '서울 강남구 남부순환로 지하 2952',
 '서울 강남구 남부순환로 지하 3104',
 '서울 강남구 일원로 지하 2',
 '서울 강남구 일원로 지하 121',
 '서울 강남구 광평로 지하 270',
 '서울 서초구 과천대로 지하 816',
 '서울 강서구 금낭화로 지하 132',
 '서울 강서구 양천로 22',
 '서울 강서구 하늘길 지하 77',
 '서울 강서구 공항대로 지하 33',
 '서울 강서구 공항대로 지하 163',
 '서울 마포구 마포대로 지하 210',
 '서울 종로구 통일

## 5. 카카오 API 활용 - 위도, 경도 검색

In [472]:
# 위도, 경도 검색 함수 정의
def lat_lon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['y'], document['x']]
        except:
            val = [0, 0]
    return val

In [473]:
lat_lon('서울 은평구 통일로 지하 723-1')

['37.6102072578868', '126.92998576213']

In [474]:
# 위도, 경도 검색 함수 반복 적용
list2 = []
for i in address_list:
    list2.append(lat_lon(i))

In [476]:
data7 = pd.DataFrame(list2, columns=['위도','경도'])
print(data7.shape)
data7.head()

(391, 2)


,위도,경도
0,37.5560572923314,126.910033925408
1,37.5491226824276,126.913544566078
2,37.5477716665856,126.922409857908
3,37.5474860706906,126.931939240491
4,37.5476683544726,126.942465251709


In [477]:
data8 = pd.concat([data6, data7], axis=1)
print(data8.shape)
data8.head()

(391, 9)


,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소,위도,경도
0,2622,망원역,Mangwon,6호선,621,망원역 6호선,서울 마포구 월드컵로 지하 77,37.5560572923314,126.910033925408
1,2623,합정역,Hapjeong,6호선,622,합정역 6호선,서울 마포구 양화로 지하 45,37.5491226824276,126.913544566078
2,2624,상수역,Sangsu,6호선,623,상수역 6호선,서울 마포구 독막로 지하 85,37.5477716665856,126.922409857908
3,2625,광흥창역,Gwangheungchang,6호선,624,광흥창역 6호선,서울 마포구 독막로 지하 165,37.5474860706906,126.931939240491
4,2626,대흥역,Daeheung,6호선,625,대흥역 6호선,서울 마포구 대흥로 지하 85,37.5476683544726,126.942465251709


## 6. 환승역 여부 컬럼 추가

In [494]:
환승역 = data8.groupby(['전철역명'])['검색키워드'].count().to_frame()
환승역 = 환승역[환승역['검색키워드'] >= 2].drop('신촌역', axis=0)   # 신촌역은 환승역이라고 볼 수 없으므로 제외
환승역

,검색키워드
전철역명,
가락시장역,2
가산디지털단지역,2
강남구청역,2
강남역,2
건대입구역,2
...,...
태릉입구역,2
합정역,2
홍대입구역,3


In [495]:
환승역.index

Index(['가락시장역', '가산디지털단지역', '강남구청역', '강남역', '건대입구역', '고속터미널역', '공덕역', '광운대역',
       '교대역', '군자역', '김포공항역', '까치산역', '노량진역', '노원역', '당산역', '대림역', '도곡역',
       '도봉산역', '동대문역', '동대문역사문화공원역', '동묘앞역', '동작역', '디지털미디어시티역', '마곡나루역',
       '망우역', '보문역', '복정역', '불광역', '사당역', '삼각지역', '상봉역', '서울역', '석계역', '석촌역',
       '선릉역', '선정릉역', '성신여대입구역', '수서역', '시청역', '신길역', '신내역', '신당역', '신도림역',
       '신설동역', '약수역', '양재역', '여의도역', '연신내역', '영등포구청역', '오금역', '옥수역', '온수역',
       '올림픽공원역', '왕십리역', '용산역', '을지로3가역', '을지로4가역', '이촌역', '잠실역', '종로3가역',
       '종합운동장역', '중랑역', '창동역', '천호역', '청구역', '청량리역', '충무로역', '충정로역', '태릉입구역',
       '합정역', '홍대입구역', '회기역', '효창공원앞역'],
      dtype='object', name='전철역명')

In [496]:
# 환승역이면 1, 아니면 0
data8['환승역'] = 0
for i in range(len(data8)):
    if data8['전철역명'][i] in 환승역.index:
        data8['환승역'][i] = 1

In [497]:
data8.head()

,전철역코드,전철역명,전철명명(영문),호선,외부코드,검색키워드,주소,위도,경도,환승역
0,2622,망원역,Mangwon,6호선,621,망원역 6호선,서울 마포구 월드컵로 지하 77,37.5560572923314,126.910033925408,0
1,2623,합정역,Hapjeong,6호선,622,합정역 6호선,서울 마포구 양화로 지하 45,37.5491226824276,126.913544566078,1
2,2624,상수역,Sangsu,6호선,623,상수역 6호선,서울 마포구 독막로 지하 85,37.5477716665856,126.922409857908,0
3,2625,광흥창역,Gwangheungchang,6호선,624,광흥창역 6호선,서울 마포구 독막로 지하 165,37.5474860706906,126.931939240491,0
4,2626,대흥역,Daeheung,6호선,625,대흥역 6호선,서울 마포구 대흥로 지하 85,37.5476683544726,126.942465251709,0


In [501]:
data8.to_csv('data/지하철_위도경도.csv', index=False)